In [1]:
from pymongo import MongoClient
import pandas as pd

import src.googla as gl
import src.geoqueries as g
import os
from dotenv import load_dotenv
import json
import requests
import src.filtering as fl
import src.geoqueries as ge
from pymongo import GEOSPHERE

load_dotenv()

True

### Para utilizar también MongoDB para hacer geoqueries, vamos a complementar el resultado obtenido en el ranking hecho anteriormente averiguando cuál de las tres oficinas con más puntuación está más cerca de una peluquería canina para Pepe, el perro de la oficina.

In [2]:
conn = MongoClient("localhost:27017")

In [3]:
db = conn.get_database("Cleaned_offices")

### Creamos el índice 2dsphere que soporta queries que calculan geometrías con la forma de la esfera terrestre. Así podemos calcular distancias y lanzar queries que nos permitan averiguar qué queda cerca de qué.

In [4]:
collection = db.get_collection("cleaned_offices")

collection.create_index([("location", '2dsphere')])




'location_2dsphere'

### Filtramos las oficinas de Canadá de nuevo para que los resultados de la geoquery sean coherentes con el análisis hecho hasta aquí.

In [5]:
can_offices = collection.find({"offices.country_code": 'CAN', "city": "Vancouver"})

In [6]:
can_offices

In [7]:
offices_list = list(can_offices)

In [8]:
db.collection.create_index([("location", '2dsphere')])

'location_2dsphere'

In [9]:
#collection.find_one()

In [10]:
#db.collection.index_information()

### Realizamos una llamada a la API de FourSquare para obtener los locales de Pet Services para obtener sus coordenadas y así averiguar si hay alguno cercano mediante la geoquery.

In [11]:
client_id = os.getenv("API_FS_CL")
secret = os.getenv("API_FS_KEY")

In [12]:
enlace = 'https://api.foursquare.com/v2/venues/explore'
pet_service = "5032897c91d4c4b30a586d69"



In [13]:
vancouver = {'type': 'Point', 'coordinates': [49.1902, -123.1837]}



In [14]:
parametros = { "client_id" : client_id,
              "client_secret" : secret,
              "v" : "20180323",
              "ll": f"{vancouver.get('coordinates')[0]},{vancouver.get('coordinates')[1]}",
              "categoryId": pet_service,
              "limit" : 100
}

In [15]:
resp = requests.get (url = enlace, params = parametros)
pet_service = json.loads(resp.text)

In [16]:
pet_service.keys()

dict_keys(['meta', 'response'])

### A partir de aquí y partiendo de las keys que ya hemos visto vamos a intentar obtener el nombre del local, la ciudad a la que pertenece (para asegurarnos de que es Vancouver así como la latitud y la longitud de cada uno).

In [17]:
res = pet_service.get('response')
#res



In [18]:
groups = res.get("groups")

In [19]:
items = groups[0].get("items")

In [20]:
from functools import reduce
import operator

In [21]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [22]:
nombre = ["venue", "name"]
ciudad = ["venue","location", "city"]
latitud = ["venue", "location","lat"]
longitud = ["venue","location","lng"]



In [23]:
obtained_data = []
for diccionario in items:
    empty = {}
    empty["nombre"] = getFromDict(diccionario,nombre)
    empty["ciudad"] = getFromDict(diccionario,ciudad)
    empty["latitud"] = getFromDict(diccionario,latitud)
    empty["longitud"] = getFromDict(diccionario,longitud)
    obtained_data.append(empty)

In [24]:
obtained_data

[{'nombre': 'PetSmart',
  'ciudad': 'Richmond',
  'latitud': 49.1930018,
  'longitud': -123.1147847},
 {'nombre': 'Tisol Richmond',
  'ciudad': 'Richmond',
  'latitud': 49.17568329890719,
  'longitud': -123.13715311773797},
 {'nombre': 'Dogsmart',
  'ciudad': 'Vancouver',
  'latitud': 49.21064804957017,
  'longitud': -123.10655834297545},
 {'nombre': 'The Groom Room',
  'ciudad': 'Vancouver',
  'latitud': 49.252255039559124,
  'longitud': -123.12742567643477}]

In [25]:
pet_services = pd.DataFrame(obtained_data)

In [26]:
pet_services

,nombre,ciudad,latitud,longitud
0,PetSmart,Richmond,49.193002,-123.114785
1,Tisol Richmond,Richmond,49.175683,-123.137153
2,Dogsmart,Vancouver,49.210648,-123.106558
3,The Groom Room,Vancouver,49.252255,-123.127426


### Aplicamos la función utilizada anteriormente para obtener la localización tipo "Point".

In [27]:
pet_services["location"] = pet_services[["latitud","longitud"]].apply(lambda x:fl.toGeoJson(x.latitud,x.longitud), axis=1)
pet_services.head()

,nombre,ciudad,latitud,longitud,location
0,PetSmart,Richmond,49.193002,-123.114785,"{'type': 'Point', 'coordinates': [-123.1147847..."
1,Tisol Richmond,Richmond,49.175683,-123.137153,"{'type': 'Point', 'coordinates': [-123.1371531..."
2,Dogsmart,Vancouver,49.210648,-123.106558,"{'type': 'Point', 'coordinates': [-123.1065583..."
3,The Groom Room,Vancouver,49.252255,-123.127426,"{'type': 'Point', 'coordinates': [-123.1274256..."


In [28]:
#pet_services.to_json("pet_services.json", orient="records")

### Pasamos el dataframe a Json y lo importamos en Mongo DB para poder trabajar con la geoquery.

In [29]:
#!mongoimport --db companies --collection pet_services --drop --jsonArray output/pet_services.json

In [30]:
conn = MongoClient("localhost:27017")

In [31]:
db1 = conn.get_database("companies")

In [32]:
collection = db1.get_collection("pet_services")

collection.create_index([("location", '2dsphere')])

'location_2dsphere'

In [33]:
collection.create_index([("geometry", '2dsphere')])

'geometry_2dsphere'

In [34]:
collection.find_one().keys()

dict_keys(['_id', 'nombre', 'ciudad', 'latitud', 'longitud', 'location'])

In [35]:
#collection.find_one()

### Importamos el CSV con el dataframe formado por las 3 oficinas con mejor puntuación para obtener sus coordenadas y poder hacer la geoquery.

In [36]:
top3_offices = pd.read_csv('output/top_3_offices.csv')

In [37]:
top3_offices

,Unnamed: 0,_id,name,category_code,founded_year,offices,total_funding,city,latitude,longitude,location,airports_2km,primary_schools_1km,night_clubs_1km,starbucks_500m,score
0,13,5fb0325c71eb61051a67b17d,Nexterra,cleantech,NaN,"{'description': 'Corporate Headquarters', 'add...",29000000,Vancouver,49.282159,-123.117650,"{'type': 'Point', 'coordinates': [-123.1176496...",5,3,20,20,12.1
1,4,5fb0325b71eb61051a67a870,BuildDirect,ecommerce,1999.0,"{'description': '', 'address1': '717 West Pend...",33000000,Vancouver,49.283935,-123.115930,"{'type': 'Point', 'coordinates': [-123.11593, ...",5,2,20,20,11.9
2,11,5fb0325c71eb61051a67b075,Ostara,cleantech,2005.0,"{'description': '', 'address1': '690 â€“ 1199 ...",45681891,Vancouver,49.288355,-123.123354,"{'type': 'Point', 'coordinates': [-123.1233545...",5,2,20,20,11.9


### Aplicamos la función "find_pet_services" que realiza una geoquery con las coordenadas de las oficinas para saber si existe algún local de servicios para perros en un radio de 4 km.

In [38]:
top3_offices['pet_services'] = top3_offices.apply(lambda x: ge.find_pet_services(db1, [x['longitude'], x['latitude']]), axis=1)

[-123.11764960000001, 49.282159]
{'location': {'$near': {'$geometry': {'type': 'Point', 'coordinates': [-123.11764960000001, 49.282159]}, '$maxDistance': 4000, '$minDistance': 0}}}
[-123.11593, 49.283935]
{'location': {'$near': {'$geometry': {'type': 'Point', 'coordinates': [-123.11593, 49.283935]}, '$maxDistance': 4000, '$minDistance': 0}}}
[-123.12335449999999, 49.28835529999999]
{'location': {'$near': {'$geometry': {'type': 'Point', 'coordinates': [-123.12335449999999, 49.28835529999999]}, '$maxDistance': 4000, '$minDistance': 0}}}


In [39]:
top3_offices['pet_services'][0]

[{'_id': ObjectId('5fb25a0503d29c7554bd5fd1'),
  'nombre': 'The Groom Room',
  'ciudad': 'Vancouver',
  'latitud': 49.2522550396,
  'longitud': -123.1274256764,
  'location': {'type': 'Point',
   'coordinates': [-123.1274256764, 49.2522550396]}}]

In [40]:
top3_offices['nearest_pet_service'] = top3_offices.apply(lambda x: gl.nearest_place(x['longitude'], x['latitude'], x['pet_services']), axis=1)

### Con esta función creamos la columna nearest_pet_service por cada una de las tres oficinas que nos muestra de la información de los locales de cuidado de mascotas, el más cercano para cada oficina.

In [41]:
top3_offices

,Unnamed: 0,_id,name,category_code,founded_year,offices,total_funding,city,latitude,longitude,location,airports_2km,primary_schools_1km,night_clubs_1km,starbucks_500m,score,pet_services,nearest_pet_service
0,13,5fb0325c71eb61051a67b17d,Nexterra,cleantech,NaN,"{'description': 'Corporate Headquarters', 'add...",29000000,Vancouver,49.282159,-123.117650,"{'type': 'Point', 'coordinates': [-123.1176496...",5,3,20,20,12.1,"[{'_id': 5fb25a0503d29c7554bd5fd1, 'nombre': '...","{'name': 'The Groom Room', 'location': [-123.1..."
1,4,5fb0325b71eb61051a67a870,BuildDirect,ecommerce,1999.0,"{'description': '', 'address1': '717 West Pend...",33000000,Vancouver,49.283935,-123.115930,"{'type': 'Point', 'coordinates': [-123.11593, ...",5,2,20,20,11.9,"[{'_id': 5fb25a0503d29c7554bd5fd1, 'nombre': '...","{'name': 'The Groom Room', 'location': [-123.1..."
2,11,5fb0325c71eb61051a67b075,Ostara,cleantech,2005.0,"{'description': '', 'address1': '690 â€“ 1199 ...",45681891,Vancouver,49.288355,-123.123354,"{'type': 'Point', 'coordinates': [-123.1233545...",5,2,20,20,11.9,[],{}


### Descartamos la 3 oficina definitivamente porque no tiene cerca una peluquería canina así que vamos a analizar de entre las otras dos, cuál tiene una más cerca:

In [42]:
top3_offices['nearest_pet_service'][0]

{'name': 'The Groom Room',
 'location': [-123.1274256764, 49.2522550396],
 'meters_to_office': 3401.0221726575073}

In [43]:
top3_offices['nearest_pet_service'][1]

{'name': 'The Groom Room',
 'location': [-123.1274256764, 49.2522550396],
 'meters_to_office': 3621.255665476988}

### Como podemos comprobar, la oficina con índice 0, la ganadora del ranking anterior es la que tiene más cerca el local de servicios para mascotas, así que sigue siendo la elegida. A continuación, en el siguiente .ipynb vamos a pintar un mapa con Folium para ver cómo quedaría el mapa con: la oficina elegida con todos los servicios así como la que ha quedado en segunda posición

## En conclusión,nos quedamos con la oficina 'Nexterra' de la categoría 'cleantech' y con las siguientes coordenadas: lat = 49.282159, lon =	-123.117650.